In [30]:
from PyPDF2 import PdfReader


In [31]:
reader = PdfReader('PSS_lunar_exploration_paper_preprint.pdf')

In [33]:
pages = reader.pages[2:37]

In [63]:
print(pages[20].extract_text())

for study on the Moon or return to the Earth. The organisms returned from these craft 
would answer many questions about the l ongevity of microorganisms in the space 
environment that will inform fields as di verse as planetary protection, for example 
allowing for an assessment of how long contaminant organisms survive on other planetary surfaces (Rummel, 2004; Glavin et al., 2004, 2010) and biogeography, for example, showing whether organisms can su rvive the conditions in interplanetary 
space and the impact conditions of landing on another planetary surface after being 
transferred from one planet to another (Clar k, 2001; Horneck et al.,  2001; Mastrapa et 
al., 2001; Burchell, 2004; Cockell et  al., 2007; Nicholson et al., 2005). 
  3.1.3 Research that advances the human exploration and settle ment of space  
 
The space environment is hostile to life and includes hard vacuum, high radiation 
(both UV and ionizing radiation) , altered gravity regimes, th e presence of biological

In [48]:
whole_article = " ".join([ page.extract_text() for page in pages])


In [ ]:
whole_article= whole_article.replace("3.1.1 Research that enhances our understandi ng of the habitability of the Earth \nthrough time","3.1.1 Research that enhances our understanding of the habitability of the Earth through time")
whole_article= whole_article.replace("3.1.2 Research that enhances our understanding of the possibili ty of life elsewhere in \nthe Universe","3.1.2 Research that enhances our understanding of the possibility of life elsewhere in the Universe")
whole_article= whole_article.replace("3.1.3 Research that advances the human exploration and settle ment of space","3.1.3 Research that advances the human exploration and settlement of space")
whole_article= whole_article.replace("2.2 The structure and compositio n of the lunar interior","2.2 The structure and composition of the lunar interior")

In [69]:
titles = [
    "1. Introduction",
    "2. Science of the Moon",
    "2.1 The Bombardment History of the Inner Solar System",
    "2.2 The structure and composition of the lunar interior",
    "2.3 The diversity of lunar crustal rocks",
    "2.4 Volatiles at the lunar poles",
    "2.5 Lunar volcanism",
    "2.6 Impact processes",
    "2.7 Regolith processes",
    "2.8 Atmospheric and dust environment",
    "3. Science on the Moon",
    "3.1 Life sciences and astrobiology",
    "3.1.1 Research that enhances our understanding of the habitability of the Earth through time",
    "3.1.2 Research that enhances our understanding of the possibility of life elsewhere in the Universe",
    "3.1.3 Research that advances the human exploration and settlement of space",
    "3.2 Human physiology and medicine",
    "3.3 Fundamental physics",
    "4. Science from the Moon – Astronomy",
    "4.1 Why Astronomy?",
    "4.2 Which astronomy?",
    "5 Conclusions",
]
for title in titles :
    assert title in whole_article, title



In [72]:
def extract_sections(whole_article,titles):
    sections = []
    part = {'title':'beginning','text':''}
    rest_text = whole_article
    for title in titles:
        text_part , rest_text = rest_text.split(title)
        part["text"] = text_part
        sections += [part.copy()]
        part = {'title':title,'text':''}

    part["text"] = rest_text
    sections += [part.copy()]
    return sections


sections = extract_sections(whole_article,titles)


In [73]:
paragraphs = []
for section in sections:
    for i,p in enumerate(section["text"].split("\n \n")):
        paragraphs += [{"section":section["title"],"number":i,"text":p}]


In [82]:
import json
with open("data.json", 'w') as json_file:
    json.dump(dict_p, json_file, indent=4)


In [2]:
import json
with open("data.json", 'r') as json_file:
    dict_p = json.load(json_file)

In [85]:
import numpy as np
p_counter = dict(zip(np.arange(1,6),np.zeros(6)))
for p in dict_p["paragraphs"]:
    if p["section"][0].isnumeric():
        i = int(p["section"][0])
        p_counter[i]+=1



In [86]:
p_counter

{1: 1.0, 2: 24.0, 3: 14.0, 4: 13.0, 5: 3.0}

In [87]:
mean_len = 0
for p in dict_p["paragraphs"]:
    mean_len += len(p["text"])
print(mean_len/len(dict_p["paragraphs"]))

1099.9642857142858


In [88]:
0.04*(576 + 196 + 169) * 700 / 1000

26.348

In [1]:
import langchain
from dotenv import load_dotenv

load_dotenv()


# Load the LLM model
chat = langchain.chat_models.ChatOpenAI(temperature=0.2, model_name='gpt-4',)


def get_basic_chat_chain(system_template: str, user_template: str) -> langchain.LLMChain:
    """
    Returns a langchain chain given an user and chat inputs

    Parameters:
        system_template (str): The system template
        user_template (str): The user template

    Returns:
        langchain.LLMChain: The langchain chain
    """

    system_prompt = langchain.prompts.SystemMessagePromptTemplate.from_template(
        system_template)

    user_prompt = langchain.prompts.HumanMessagePromptTemplate.from_template(
        user_template)

    chat_prompt = langchain.prompts.ChatPromptTemplate.from_messages([
        system_prompt,
        user_prompt
    ])

    return langchain.LLMChain(llm=chat, prompt=chat_prompt)



In [25]:
system_promt = """
Your are an assistant and you are creating a knowledge graph from a scientific article the talks about the moon exploration.
You will be given as input two pieces of text from the article. 

You have to identify the main concepts of each piece of text.
And depending on what is said in both texts you need to identify possible relations between the concepts.


You will return the answer as a json object:


    'textA_concepts' : ['concept1','concept2','concept3',...],
    'textB_concepts' : ['concept4','concept5','concept6',...],
    'relations' : [
        ('concept9','concept10','relation1'),
        ('concept9','concept2','relation2'),
    ],


You can select as many concepts as you want but they have to be relevant.
You can etablish as many relations as you want.

"""
user_prompt = """
## INPUTS:

#TEXT A
{T1}

#TEXT B
{T2}
"""

In [27]:
chain = get_basic_chat_chain(system_promt,user_prompt)

In [28]:
response = chain.run(T1=dict_p["paragraphs"][18]["text"],T2=dict_p["paragraphs"][10]["text"])

In [30]:
print(response)

## OUTPUT:

    {
    'textA_concepts' : ['remote sensing measurements', 'concentration of polar ice', 'high latitude surficial OH/H2O', 'in situ measurements', 'landed spacecraft', 'permanently shadowed environments', 'high latitude environments', 'temperatures below 40 K', 'human exploration', 'robotic vehicles', 'nuclear power source', 'long-lived batteries', 'penetrator-based system', 'MoonLITE', 'LunarNet mission concepts', 'non-permanently shadowed polar localities', 'solar-powered robotic exploration', 'ESA’s Lunar Lander', 'Lunar Beagle', 'measurements of volatiles', 'characterization of polar volatiles', 'lunar geology', 'human exploration', 'exploitable quantities of volatiles'],
    'textB_concepts' : ['thermal perturbation', 'autonomous lander', 'heat flow measurements', 'landing site', 'thermal probes', 'human operated drill', 'Apollo astronauts', 'human operated portable magnetometers', 'Apollo 14 and 16 missions', 'surface magnetic fields', 'kilometer scale distances', '

In [22]:
system_promt = """
Your are an assistant and you are creating a knowledge graph from a scientific article the talks about the moon exploration.
You will be given as input a piece of text from the article. 
Your job is to identify the main concepts to use them as nodes in our knowledge graph. 
Then you have to identify the relations between each concept.

You will return the answer as a JSON object with the keys:
    - "nodes" and the value as a list of strings ['concept1','concept2','concept3',...].
    - "relations" and the value as a list of lists [['concept9','concept10','relation1'],['concept9','concept2','relation2']...]
You have to only return the JSON object.
Your answer begins with a left curly bracket and ends with a right curly bracket.
No preface text.

"""
user_prompt = """
## TEXT:
{text}

"""

In [23]:
chain = get_basic_chat_chain(system_promt,user_prompt)

In [24]:
for p in dict_p["paragraphs"][0]
text=dict_p["paragraphs"][0]["text"]
response = chain.run(text=text)

In [25]:
print(response)

{"nodes": ["lunar geological record", "Solar System", "origin and evolution of the Earth-Moon system", "geological evolution of rocky planets", "near-Earth cosmic environment", "lunar surface", "research in astronomy", "astrobiology", "fundamental physics", "life sciences", "human physiology and medicine", "lunar science objectives", "lunar surface exploration", "scientific instruments", "surface of the Moon", "robotic exploration", "geophysical networks", "radio telescopes", "human operations on the lunar surface", "human lunar exploration programme", "Global Exploration Roadmap", "Lunar geology", "Lunar geophysics", "Lunar astronomy", "Space exploration", "Space life sciences", "Space medicine"],
"relations": [["lunar geological record", "Solar System", "provides information about"], ["lunar geological record", "origin and evolution of the Earth-Moon system", "provides information about"], ["lunar geological record", "geological evolution of rocky planets", "provides information abou

In [26]:
graph = json.loads(response)

In [28]:
with open("graph_abstract","w") as f:
    json.dump(graph, f, indent=4)

In [13]:
from pprint import pprint
pprint(response)

('{"nodes": ["lunar geological record", "Solar System", "origin and evolution '
 'of the Earth-Moon system", "geological evolution of rocky planets", '
 '"near-Earth cosmic environment", "lunar surface", "research in astronomy", '
 '"astrobiology", "fundamental physics", "life sciences", "human physiology '
 'and medicine", "lunar science objectives", "lunar surface exploration", '
 '"scientific instruments", "samples from the Moon", "robotic operations", '
 '"geophysical networks", "radio telescopes", "human operations on the lunar '
 'surface", "robotic surface exploration of the Moon", "human lunar '
 'exploration programme", "Global Exploration Roadmap", "Lunar geology", '
 '"Lunar geophysics", "Lunar astronomy", "Space exploration", "Space life '
 'sciences", "Space medicine"],\n'
 '"relations": [("lunar geological record", "Solar System", "tells about the '
 'history of"), ("lunar geological record", "origin and evolution of the '
 'Earth-Moon system", "tells about"), ("lunar geo